# Transfer Learning of ResNet50

The purpose of this notebook is to reimplement the results of the following paper:

* https://link.springer.com/article/10.1007%2Fs00521-020-05437-x 



Data source: 

* https://www.kaggle.com/mloey1/covid19-chest-ct-image-augmentation-gan-dataset 

In [38]:
import pandas as pd
import matplotlib.pyplot as plt
import sys
sys.path.append("../")
from covidct.dataset import *

path = '../data/'

train_data = CovidCTDataset(path, with_aug=False, with_cgan=False, split = 'train')
val_data = CovidCTDataset(path, with_aug=False, with_cgan=False, split = 'val')
test_data = CovidCTDataset(path, with_aug=False, with_cgan=False, split = 'test')

print(len(train_data))
print(len(val_data))
print(len(test_data))

425
118
199


# Loading ResNet50 pretrained model

https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html  

In [50]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

resnet = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True)

num_ftrs = resnet.fc.in_features

resnet.fc = torch.nn.Linear(2048, 2)

resnet

Using cache found in C:\Users\Alex/.cache\torch\hub\pytorch_vision_v0.10.0


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

# Transfer learning with COVID-CT data to ResNet50 model

## define model training

In [52]:
from tqdm import tqdm
import copy

def train_resnet(model, data, criterion, optimizer, n_epochs):
    '''f'''

    val_acc_history = []
    best_acc = 0.0

    # iterating epochs
    for epoch in tqdm(range(n_epochs)):
        for phase in ['train', 'val']:
            if phase == 'train':
                resnet.train()
            else:
                resnet.eval()

            running_loss = 0.0
            running_corrects = 0

            # iterating data
            for x, y in data[phase]:
                inputs = x.to(device)
                labels = y.to(device)

                optimizer.zero_grad()

                # forward
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                    _, predictions = torch.max(outputs, 1)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # update running values
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(predictions == labels.data)

            epoch_loss = running_loss / len(data[phase].dataset)
            epoch_acc = running_corrects.double() / len(data[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

            if phase == 'val':
                val_acc_history.append(epoch_acc)
        print()

    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history


def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_gradd = False

## creating optimizer

In [54]:
# model to device
resnet = resnet.to(device)

feature_extract = True

# Gather the parameters to be optimized/updated in this run. If we are
#  finetuning we will be updating all parameters. However, if we are
#  doing feature extract method, we will only update the parameters
#  that we have just initialized, i.e. the parameters with requires_grad
#  is True.

params_to_update = resnet.parameters()
print('Parameters to learn:')

if feature_extract:
    params_to_update = []
    for name, param in resnet.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print('\t', name)
else:
    for name, param in resnet.named_parameters():
        if param.requires_grad == True:
            print('\t', name)

optimizer_ft = torch.optim.Adam(params_to_update, lr = 0.001)

Parameters to learn:
	 conv1.weight
	 bn1.weight
	 bn1.bias
	 layer1.0.conv1.weight
	 layer1.0.bn1.weight
	 layer1.0.bn1.bias
	 layer1.0.conv2.weight
	 layer1.0.bn2.weight
	 layer1.0.bn2.bias
	 layer1.0.conv3.weight
	 layer1.0.bn3.weight
	 layer1.0.bn3.bias
	 layer1.0.downsample.0.weight
	 layer1.0.downsample.1.weight
	 layer1.0.downsample.1.bias
	 layer1.1.conv1.weight
	 layer1.1.bn1.weight
	 layer1.1.bn1.bias
	 layer1.1.conv2.weight
	 layer1.1.bn2.weight
	 layer1.1.bn2.bias
	 layer1.1.conv3.weight
	 layer1.1.bn3.weight
	 layer1.1.bn3.bias
	 layer1.2.conv1.weight
	 layer1.2.bn1.weight
	 layer1.2.bn1.bias
	 layer1.2.conv2.weight
	 layer1.2.bn2.weight
	 layer1.2.bn2.bias
	 layer1.2.conv3.weight
	 layer1.2.bn3.weight
	 layer1.2.bn3.bias
	 layer2.0.conv1.weight
	 layer2.0.bn1.weight
	 layer2.0.bn1.bias
	 layer2.0.conv2.weight
	 layer2.0.bn2.weight
	 layer2.0.bn2.bias
	 layer2.0.conv3.weight
	 layer2.0.bn3.weight
	 layer2.0.bn3.bias
	 layer2.0.downsample.0.weight
	 layer2.0.downsample.1.we

In [55]:
# hyperparameters
n_epochs = 50
batch_size = 32

data = {'train': torch.utils.data.DataLoader(train_data, shuffle = True, batch_size = batch_size)
        ,'val': torch.utils.data.DataLoader(val_data, shuffle = True, batch_size = batch_size)
        ,'test': torch.utils.data.DataLoader(test_data, shuffle = True, batch_size = batch_size)}

In [56]:
criterion = torch.nn.CrossEntropyLoss()

resnet, hist = train_resnet(resnet, data, criterion, optimizer_ft, n_epochs)

  0%|          | 0/50 [00:00<?, ?it/s]

train Loss: 0.6852 Acc: 0.7012


  2%|▏         | 1/50 [03:15<2:39:18, 195.07s/it]

val Loss: 27.2574 Acc: 0.6186

train Loss: 0.5757 Acc: 0.7341


  4%|▍         | 2/50 [06:32<2:37:15, 196.57s/it]

...

val Loss: 1.1489 Acc: 0.6864

train Loss: 0.0677 Acc: 0.9788


 98%|█████████▊| 49/50 [3:18:54<03:07, 187.91s/it]

val Loss: 0.7642 Acc: 0.6864

train Loss: 0.0250 Acc: 0.9953


100%|██████████| 50/50 [3:22:11<00:00, 242.62s/it]

val Loss: 0.9440 Acc: 0.7458

Best val Acc: 0.779661


## Testing model on test dataset

In [91]:
with torch.no_grad():
    resnet.eval()
    y_pred = []
    y_true = []
    
    for img, lab in data['test']:
        batch_pred = resnet(img)
        for i, (y0, y1) in enumerate(batch_pred):
            if y0 > y1:
                pred = 0
            else:
                pred = 1
            y_pred.append(pred)
            y_true.append(lab[i].item())

print(y_pred)
print(y_true)

[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0]
[0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1,

In [109]:
import sklearn.metrics as metrics

print('true class')
print(metrics.confusion_matrix(y_true, y_pred))
print('--------')
print(f'accuracy: {metrics.accuracy_score(y_true, y_pred)}')
print(f'precision: {metrics.precision_score(y_true, y_pred)}')
print(f'recall: {metrics.recall_score(y_true, y_pred)}')
print(f'f1 score: {metrics.f1_score(y_true, y_pred)}')

true class
[[83 22]
 [40 54]]
--------
accuracy: 0.6884422110552764
precision: 0.7105263157894737
recall: 0.574468085106383
f1 score: 0.6352941176470589


## Saving model to repo

In [63]:
torch.save(resnet.state_dict(), '../models/ResNet50_raw/model.wts')